In [1]:
from human_annotator_with_hint import MySQLDataHandler, Annotation, ModelTraining, ActiveLearning

In [2]:
from datasets import Dataset, Features, Value,load_dataset,DatasetDict


In [3]:
import pandas as pd

In [4]:
uname = 'zijianan'

In [5]:
ori_table_name = 'blm_tweets'

In [6]:
human_annotated_table_name = 'blm_annotated_tweets'

In [7]:
db_handler = MySQLDataHandler('localhost', 'zijianan', '#######', '5_category')

In [8]:
save_path = '/projects/academic/kjoseph/zijian/'+uname+'_'

In [9]:
labels = ['pos','neg','unknown']

### random sample

In [10]:
query = f'''
SELECT * FROM {ori_table_name} ORDER BY RAND() LIMIT 50;
'''
random_blm_data = db_handler.fetch_data(query=query)

In [11]:
random_blm_data

,id,rt_id,rt_uid,modeling_text
0,327408,763915332431941632,347627434,so wrong\nnfl denies dallas cowboys’ req to ho...
1,426399,1275114784787247104,47839801,"get in losers, we’re dismantling the patriarchy."
2,3544479,1284228067561558018,932630991298007041,the us equity markets = absolute units
3,838469,1311615252748996608,16041234,trump administration instructed law enforcemen...
4,3229052,1656723610743222273,55060090,mitt romney condemns trump for offering pardon...
5,4331812,909880965475868672,22637974,"yale joins list of universities replacing ""fre..."
6,723406,1670135823684059137,55330244,nova scotia police thwart 3 arson attacks as m...
7,976267,1348348496018104322,1201348645225930752,the maga cult terrorists are vowing to return ...
8,507700,1399007405468532743,170254080,after the “socialist”-branded no evil foods bu...
9,3705769,301531740466470912,14752085,rt @michaelianblack: pay the people who write ...


In [12]:
random_blm_data = pd.read_csv('/projects/academic/kjoseph/zijian/blm_test.csv')

In [13]:

annotations = Annotation.run_annotation(random_blm_data,labels,'modeling_text')

HTML(value='0 of 50 Examples annotated, Current Position: 0 ')

Output()

In [17]:
random_blm_data['label'] = annotations['label']

In [14]:
validate_data = pd.read_csv('/projects/academic/kjoseph/zijian/negative_tweet_examples.csv')

In [15]:
rt_ids = validate_data['rt_id'].tolist()
rt_ids_str = ', '.join([str(id) for id in rt_ids])

In [16]:
query = f"SELECT * FROM {ori_table_name} WHERE rt_id IN ({rt_ids_str}) ORDER BY RAND() LIMIT 200;"
neg_vali = db_handler.fetch_data(query=query)

In [17]:
query = f"SELECT * FROM {ori_table_name} WHERE rt_id NOT IN ({rt_ids_str}) ORDER BY RAND() LIMIT 200;"
pos_vali = db_handler.fetch_data(query=query)

In [18]:
neg_vali['label'] = 'neg'
pos_vali['label'] = 'pos'

In [19]:
random_blm_data_vali = pd.concat([neg_vali,pos_vali]).reset_index(drop=True)

In [16]:
# vali_per = 0.4
# random_blm_data_vali = random_blm_data.groupby('label').apply(lambda x: x.sample(frac=vali_per, random_state=42)).reset_index(drop=True)


In [20]:
dataset = Dataset.from_pandas(random_blm_data[random_blm_data['label'].isin(['neg','pos'])][['modeling_text','label']])

In [21]:
dataset = DatasetDict({
    'train': dataset  # Set the 'train' split
})

In [22]:
dataset['validation'] = Dataset.from_pandas(random_blm_data_vali[['modeling_text','label']])

In [23]:
dataset["test"] = dataset["validation"]

In [24]:
save_path

'/projects/academic/kjoseph/zijian/zijianan_'

In [25]:
save_path = save_path + "blm_random_0"

In [31]:
import os
os.environ['WANDB_DISABLED'] = "true"

In [32]:

training_instance = ModelTraining(dataset=dataset, \
                                  model_name_or_path='launch/POLITICS', \
                                  label_column_name='label', \
                                  text_column_name='modeling_text',\
                                  save_path=save_path, \
                                  num_train_epochs=10)
model_0 = training_instance.train_model()
model_0.to('cpu')
import torch
torch.cuda.empty_cache()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
[WARNING|integration_utils.py:82] 2024-05-13 15:54:07,244 >> Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


05/13/2024 15:54:07 - WARNING - fastfit.train - Process rank: 0, device: cuda:0, n_gpu: 2distributed training: True, 16-bits training: True


/projects/academic/kjoseph/zijian/pytorch/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
[WARNING|modeling_utils.py:4172] 2024-05-13 15:54:11,151 >> Some weights of RobertaModel were not initialized from the model checkpoint at launch/POLITICS and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Running tokenizer on dataset to infer max length for both query and document:   0%|          | 0/29 [00:00<?, …

Running tokenizer on dataset to infer max length for both query and document:   0%|          | 0/400 [00:00<?,…

...

Running tokenizer on dataset:   0%|          | 0/400 [00:00<?, ? examples/s]

/projects/academic/kjoseph/zijian/pytorch/lib/python3.9/site-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/cvmfs/soft.ccr.buffalo.edu/versions/2023.01/easybuild/software/avx512/MPI/gcc/11.2.0/openmpi/4.1.1/pytorch/1.13.1-CUDA-11.8.0/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
[WARNING|modeling_utils.py:1188] 2024-05-13 15:54:2

Step,Training Loss


***** train metrics *****
  epoch                    =       10.0
  total_flos               =        0GF
  train_loss               =      4.015
  train_runtime            = 0:00:04.97
  train_samples            =         29
  train_samples_per_second =     58.298
  train_steps_per_second   =       2.01


***** eval metrics *****
  epoch                   =       10.0
  eval_accuracy           =     0.5475
  eval_loss               =      5.442
  eval_runtime            = 0:00:01.04
  eval_samples            =        400
  eval_samples_per_second =    383.723
  eval_steps_per_second   =      6.715
{'eval_loss': 5.441959857940674, 'eval_accuracy': 0.5475, 'eval_runtime': 1.0424, 'eval_samples_per_second': 383.723, 'eval_steps_per_second': 6.715, 'epoch': 10.0, 'eval_samples': 400}
Accuracy: 54.8%


### active learning

In [24]:

# query = f'''
# SELECT * FROM {ori_table_name} ORDER BY RAND() LIMIT 1000;
# '''
# inference_data = db_handler.fetch_data(query=query)

In [33]:
inference_data = random_blm_data_vali.drop('label',axis=1)

In [36]:
save_path = '/projects/academic/kjoseph/zijian/zijianan_blm_random_'

In [37]:
active_learning_instance = ActiveLearning(model_name_or_path=save_path, \
                                  label_column_name='label', \
                                  text_column_name='modeling_text',\
                                  theta=10,
                                  i=3,
                                  vali_data=dataset["validation"],
                                  inference_datasets=inference_data,
                                  tokenizer_name='roberta-large',
                                  uname=uname,
                                  labels=['neg','pos'])

/projects/academic/kjoseph/zijian/pytorch/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

[ERROR|base.py:1089] 2024-05-13 15:55:25,544 >> The model 'FastFit' is not supported for text-classification. Supported models are ['AlbertForSequenceClassification', 'BartForSequenceClassification', 'BertForSequenceClassification', 'BigBirdForSequenceClassification', 'BigBirdPegasusForSequenceClassification', 'BioGptForSequenceClassification', 'BloomForSequenceClassification', 'CamembertForSequenceClassification', 'CanineForSequenceClassification', 'LlamaForSequenceClassification', 'ConvBertForSequenceClassification', 'CTRLForSequenceClassification', 'Data2VecTextForSequenceClassification', 'DebertaForSequenceClassification', 'DebertaV2ForSequenceClassification', 'DistilBertForSequenceClassification', 'ElectraForSequenceClassification', 'ErnieForSequenceClassification', 'ErnieMForSequenceClassification', 'EsmForSequenceClassification', 'FalconForSequenceClassification', 'FlaubertForSequenceClassification', 'FNetForSequenceClassification', 'FunnelForSequenceClassification', 'GemmaForSe

In [ ]:
t1 = active_learning_instance.perform_epochs()

Epoch 1/3


[ERROR|base.py:1089] 2024-05-13 15:55:29,468 >> The model 'FastFit' is not supported for text-classification. Supported models are ['AlbertForSequenceClassification', 'BartForSequenceClassification', 'BertForSequenceClassification', 'BigBirdForSequenceClassification', 'BigBirdPegasusForSequenceClassification', 'BioGptForSequenceClassification', 'BloomForSequenceClassification', 'CamembertForSequenceClassification', 'CanineForSequenceClassification', 'LlamaForSequenceClassification', 'ConvBertForSequenceClassification', 'CTRLForSequenceClassification', 'Data2VecTextForSequenceClassification', 'DebertaForSequenceClassification', 'DebertaV2ForSequenceClassification', 'DistilBertForSequenceClassification', 'ElectraForSequenceClassification', 'ErnieForSequenceClassification', 'ErnieMForSequenceClassification', 'EsmForSequenceClassification', 'FalconForSequenceClassification', 'FlaubertForSequenceClassification', 'FNetForSequenceClassification', 'FunnelForSequenceClassification', 'GemmaForSe

HTML(value='0 of 10 Examples annotated, Current Position: 0 ')

Output()

Annotation done.


...

***** train metrics *****
  epoch                    =       10.0
  total_flos               =        0GF
  train_loss               =     2.9936
  train_runtime            = 0:00:02.08
  train_samples            =         10
  train_samples_per_second =     47.889
  train_steps_per_second   =      4.789


***** eval metrics *****
  epoch                   =       10.0
  eval_accuracy           =      0.505
  eval_loss               =     5.1015
  eval_runtime            = 0:00:00.94
  eval_samples            =        400
  eval_samples_per_second =    421.282
  eval_steps_per_second   =      7.372
{'eval_loss': 5.1015143394470215, 'eval_accuracy': 0.505, 'eval_runtime': 0.9495, 'eval_samples_per_second': 421.282, 'eval_steps_per_second': 7.372, 'epoch': 10.0, 'eval_samples': 400}
Accuracy: 50.5%
Epoch 2/3


[ERROR|base.py:1089] 2024-05-13 15:55:53,260 >> The model 'FastFit' is not supported for text-classification. Supported models are ['AlbertForSequenceClassification', 'BartForSequenceClassification', 'BertForSequenceClassification', 'BigBirdForSequenceClassification', 'BigBirdPegasusForSequenceClassification', 'BioGptForSequenceClassification', 'BloomForSequenceClassification', 'CamembertForSequenceClassification', 'CanineForSequenceClassification', 'LlamaForSequenceClassification', 'ConvBertForSequenceClassification', 'CTRLForSequenceClassification', 'Data2VecTextForSequenceClassification', 'DebertaForSequenceClassification', 'DebertaV2ForSequenceClassification', 'DistilBertForSequenceClassification', 'ElectraForSequenceClassification', 'ErnieForSequenceClassification', 'ErnieMForSequenceClassification', 'EsmForSequenceClassification', 'FalconForSequenceClassification', 'FlaubertForSequenceClassification', 'FNetForSequenceClassification', 'FunnelForSequenceClassification', 'GemmaForSe

HTML(value='0 of 10 Examples annotated, Current Position: 0 ')

Output()